In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
from __future__ import print_function
from util_functions import process_files

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [12]:
batch_size = 32
num_classes = 2
epochs = 100

# load the data
x_train, y_train, f_train = process_files(dataset='training', features=['mfccs', 'mfcc_deltas'], shape='stacked')
x_test, y_test, f_test = process_files(dataset='test', features=['mfccs', 'mfcc_deltas'], shape='stacked')

# input image dimensions
input_d = x_train.shape[1] #Depth
input_h = x_train.shape[2] #Height
input_w = x_train.shape[3] #Width

#Reshaping to feed to network
x_train = x_train.reshape(x_train.shape[0], input_h, input_w, input_d)
x_test = x_test.reshape(x_test.shape[0], input_h, input_w, input_d)
input_shape = (input_h, input_w, input_d)

#Making them floats for TF
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# from the results in 10 epochs we can see that the other learning rates are just too big. 
# Hence only this one will be used
learning_rate = 0.0001
dropout_rate = 0.4

model = Sequential()
model.add(Conv2D(64, kernel_size=(1, 1), activation='tanh', input_shape=input_shape))
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(dropout_rate))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Nadam(lr=learning_rate),
              metrics=['accuracy'])

tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,  
      write_graph=True, write_images=True)

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=2,
#           validation_split=0.1,
          callbacks=[tbCallBack])

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])        

print('----------------------------------------------------\n')    

Epoch 1/100
 - 2s - loss: 0.5936 - acc: 0.7107
Epoch 2/100
 - 2s - loss: 0.4280 - acc: 0.8182
Epoch 3/100
 - 2s - loss: 0.3356 - acc: 0.8581
Epoch 4/100
 - 2s - loss: 0.3042 - acc: 0.8767
Epoch 5/100
 - 2s - loss: 0.2493 - acc: 0.8981
Epoch 6/100
 - 2s - loss: 0.2069 - acc: 0.9167
Epoch 7/100
 - 2s - loss: 0.1793 - acc: 0.9311
Epoch 8/100
 - 2s - loss: 0.1479 - acc: 0.9470
Epoch 9/100
 - 2s - loss: 0.1192 - acc: 0.9614
Epoch 10/100
 - 2s - loss: 0.1058 - acc: 0.9635
Epoch 11/100
 - 2s - loss: 0.0651 - acc: 0.9752
Epoch 12/100
 - 2s - loss: 0.0761 - acc: 0.9738
Epoch 13/100
 - 2s - loss: 0.0670 - acc: 0.9725
Epoch 14/100
 - 2s - loss: 0.0417 - acc: 0.9842
Epoch 15/100
 - 2s - loss: 0.0414 - acc: 0.9862
Epoch 16/100
 - 2s - loss: 0.0305 - acc: 0.9904
Epoch 17/100
 - 2s - loss: 0.0272 - acc: 0.9904
Epoch 18/100
 - 2s - loss: 0.0262 - acc: 0.9910
Epoch 19/100
 - 2s - loss: 0.0333 - acc: 0.9883
Epoch 20/100
 - 2s - loss: 0.0318 - acc: 0.9876
Epoch 21/100
 - 2s - loss: 0.0251 - acc: 0.9924
E

In [11]:
# x_train